In [7]:
import os
import lmdb 
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [8]:
y_train = ["壹萬元整","陸萬元整","參拾貳萬肆仟元整","伍萬元整","壹萬元整","參仟肆佰陸拾肆元整","參仟伍佰壹拾參元整","參仟肆佰貳拾元整","壹萬元整","貳拾參萬陸仟元整"]

In [9]:
data = []
for g in os.listdir('/Users/chienan/Job/CNN/cut/'):
    data.append(plt.imread('./cut/%s'%(g)))
data = data[:10]
X = np.array([i.reshape(1,32,100) for i in data])/255

In [10]:
def writeCache(env, cache):
    with env.begin(write=True) as txn:
        for k, v in cache.iteritems():
            txn.put(k, v)

In [11]:
def read_text(path):
    with open(path) as f:
        text = f.read()
    text = text.strip()
    return text

In [12]:
text_file = y_train
image_file = os.listdir('/Users/chienan/Job/CNN/cut/')[:10]
outputPath_train = '/Users/chienan/Job/CNN/lmdb/'

In [13]:
env_train = lmdb.open(outputPath_train, map_size=1099511627776)

In [14]:
# env = lmdb.open(
#             outputPath,
#             max_readers=1,
#             readonly=False,
#             lock=False,
#             readahead=False,
#             meminit=False)

In [16]:
cnt = 1
cache = {}
nSamples = len(image_file)
for i in range(nSamples):
    imagePath = '/Users/chienan/Job/CNN/cut/'+image_file[i]
    label = text_file[i]
    if not os.path.exists(imagePath):
        print('%s does not exist' % imagePath)
        continue
    with open(imagePath,'rb') as f:
        image = f.read()
    imageKey = 'image-%09d' % cnt
    labelKey = 'label-%09d' % cnt
    cache[imageKey] = image
    cache[labelKey] = label.encode()
    
    if cnt % 1 == 0:
        with env_train.begin(write=True) as txn:
            for k, v in zip(cache.keys(),cache.values()):                
                txn.put(k.encode(), v)                
        cache = {}
        print('Written %d / %d' % (cnt, nSamples))
    cnt += 1
nSamples = cnt-1

Written 1 / 10
Written 2 / 10
Written 3 / 10
Written 4 / 10
Written 5 / 10
Written 6 / 10
Written 7 / 10
Written 8 / 10
Written 9 / 10
Written 10 / 10


In [17]:
cache['num-samples'] = str(nSamples)

with env_train.begin(write=True) as txn:
    for k, v in zip(cache.keys(),cache.values()):
        txn.put(k.encode(), v.encode())
print('Created dataset with %d samples' % nSamples)

Created dataset with 10 samples


In [18]:
with env_train.begin(write=False) as txn:
    t= txn.get("label-000000009".encode())

In [21]:
t.decode()

'壹萬元整'